In [1]:
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_volumetric"

In [3]:
# Compute population centroid
circuit = obi.Circuit(name="nbS1-O1", path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"))
c = circuit.sonata_circuit
nodes = c.nodes[circuit.default_population_name]
population_centroid = nodes.positions().mean()

In [4]:
# nbS1-O1
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")

# Load x/y/z positions from connectivity matrix and select best neuron in each layer
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

xyz_selection = []
for _lay in layers:
    opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    xyz_selection.append(ntab.loc[node_id][["x", "y", "z"]].astype(float).to_list())
    assert ntab.loc[node_id]["layer"] == str(_lay), "ERROR: Layer mismatch!"
xyz_selection = pd.DataFrame(xyz_selection, columns=["x", "y", "z"], index=layers) - population_centroid  # Relative to population centroid
xyz_selection.index.name = "layer"

In [5]:
# Set up volumetric neuron sets
N = 10
volumetric_neuron_sets = obi.VolumetricCountNeuronSet(ox=xyz_selection["x"].to_list(), oy=xyz_selection["y"].to_list(), oz=xyz_selection["z"].to_list(), n=N,
                                                      property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={}))
virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "VPM") if _lay == 1 else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers]

In [6]:
# # nbS1-O1 - PLOTTING
# # cmat_O1 = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-O1", "connectivity_matrix.h5"))

# # Load x/y/z positions from connectivity matrix
# base_name = "nbS1"
# plt.figure()
# for hex_sel in [f"hex{_i}" for _i in range(7)]:
#     cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
#     plt.plot(cmat.vertices["y"], cmat.vertices["z"], '.')
# # plt.plot(cmat_O1.vertices["y"], cmat_O1.vertices["z"], ',k')
# # plt.plot(xyz_selection["y"], xyz_selection["z"], "xk")
# plt.show()

# # df_sel[("hex0", 1)]

In [7]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    # neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists),
                    neuron_set=volumetric_neuron_sets
)

coupled_scan = obi.CoupledScan(form=circuit_extractions_form, output_root=output_root, coordinate_directory_option="ZERO_INDEX")

In [8]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
coupled_scan.coordinate_instances(display=True)

[2025-06-11 18:00:39,975] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-11 18:00:39,977] INFO: neuron_set.ox: [723.5147043713164, 456.74954380015106, 496.86043927062155, 164.0458458805274, -111.82257508700832, -367.728748031805]
[2025-06-11 18:00:39,978] INFO: neuron_set.oy: [-136.48814337810632, -204.13295856910258, -65.5996708227492, -228.95787402852488, -134.55755720016236, 225.80812622163035]
[2025-06-11 18:00:39,979] INFO: neuron_set.oz: [559.3665277368043, 544.6067321151927, 438.3645986873328, 253.1818605500871, -259.3746148238397, -453.693623150381]
[2025-06-11 18:00:39,980] INFO: initialize.virtual_sources_to_ignore: [('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'VPM'), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'ox'], value=723.5147043713164), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'oy'], value=-136.48814337810632), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'oz'], value=559.3665277368043), SingleValueScanParam(type='SingleValueScanParam', location_list=['initialize', 'virtual_sources_to_ignore'], value=('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'VPM'))], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=VolumetricCountNeuronSet(type='VolumetricCountNeuronSet', simulation_level_name=None, random_sample=None, random_seed=0, property_filter=NoFilter, node_sets=(), ox=723.5147043713164, oy=

In [9]:
coupled_scan.execute(processing_method='run')

Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 845.68it/s]

Copied 10 morphologies into container (0 already existed)
Copying 6 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 829.24it/s]

...

Copied 10 morphologies into container (0 already existed)
Copying 1 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 933.98it/s]

Copied 10 morphologies into container (0 already existed)
Copying 1 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE
[2025-06-11 18:15:31,898] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [24]:
# Check circuits
from bluepysnap import Circuit

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    print(f"L{layers[_idx]}: {c.nodes['S1nonbarrel_neurons'].size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {np.unique(c.nodes['S1nonbarrel_neurons'].get(properties=['mtype'])).tolist()}")


L1: 10 neurons, 234 synapses, m-types: ['L1_HAC', 'L1_LAC', 'L1_NGC-DA', 'L1_NGC-SA', 'L1_SAC']
L2: 10 neurons, 186 synapses, m-types: ['L23_CHC', 'L23_LBC', 'L23_NBC', 'L2_TPC:B', 'L3_TPC:A']
L3: 10 neurons, 96 synapses, m-types: ['L23_NBC', 'L3_TPC:A', 'L3_TPC:C']
L4: 10 neurons, 48 synapses, m-types: ['L4_SSC', 'L4_TPC', 'L4_UPC']
L5: 10 neurons, 69 synapses, m-types: ['L5_TPC:A', 'L5_TPC:B', 'L5_TPC:C']
L6: 10 neurons, 92 synapses, m-types: ['L6_BPC', 'L6_IPC', 'L6_TPC:A', 'L6_TPC:C']
